In [25]:
pip install langchain langchain-core langchain-community langchain_ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\lonyi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [26]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gemma3")

llm.invoke("The first man on the moon was ...")

'Neil Armstrong! \n\nHe famously said, "That\'s one small step for [a] man, one giant leap for mankind." 😊 \n\nDo you want to know more about the Apollo 11 mission?'

In [27]:
for chunk in llm.stream("The first man on the moon was ..."):
    print(chunk, end="|", flush=True)

Neil| Armstrong|!| |

|He| was| the| first| person| to| step| onto| the| Moon| on| July| |2|0|,| |1|9|6|9|,| as| part| of| the| Apollo| |1|1| mission|.| |

|Do| you| want| to| know| more| about| the| mission| or| Neil| Armstrong|?||

In [28]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="gemma3")

chat_model.invoke("Who was the first man on the moon?")

AIMessage(content='The first man to walk on the moon was **Neil Armstrong**. \n\nHe took that historic step on July 20, 1969, as part of the Apollo 11 mission. \n\nWhile Buzz Aldrin was the second person to walk on the moon that day, Armstrong is widely recognized as the first. 😊 \n\nWould you like to know more about the Apollo 11 mission or Neil Armstrong?', additional_kwargs={}, response_metadata={'model': 'gemma3', 'created_at': '2025-05-06T19:23:45.2518542Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1272176300, 'load_duration': 24011900, 'prompt_eval_count': 18, 'prompt_eval_duration': 22195700, 'eval_count': 89, 'eval_duration': 1225968700, 'model_name': 'gemma3'}, id='run--443f4cad-6770-4c7c-923a-ebc35ea33485-0', usage_metadata={'input_tokens': 18, 'output_tokens': 89, 'total_tokens': 107})

In [29]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [30]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [31]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

C:\Users\lonyi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [32]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

AttributeError: 'str' object has no attribute 'content'